<a href="https://colab.research.google.com/github/leorumma/arvore-decisao-v2/blob/main/arvore-decisao-parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from seaborn import heatmap
import matplotlib.pyplot as plt
import random

#criar os gráficos das árvores de decisão
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image  
import pydotplus

In [ ]:
def generate_decision_tree_png(decision_tree_clf, feature_names, file_name = 'arvore1.png'):
    # tem que usar feature_names = one_hot_data.columns pois feature_names = feature_cols tem menos atributos
    # pois o one-hot acrescenta mais
    dot_data = StringIO()
    export_graphviz(decision_tree_clf, out_file=dot_data,  
                    filled=True, rounded=True,
                    special_characters=True,feature_names = feature_names, class_names=['No','Yes'])
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    graph.write_png(file_name)
    #clf_image_png = graph.create_png()
    
    #Image(clf_image_png)

In [ ]:
#link para o dataset: 
DATASET_URL = 'Online_Payments_Fraud_Detection_Dataset.csv'
dataset = pd.read_csv(DATASET_URL)
original_dataset = dataset.copy()

In [ ]:
dataset.head()

In [ ]:
#Iniciando análise exploratória dos dados

In [ ]:
#Sem valores não preenchidos nas colunas
print(len(dataset))
print(len(dataset.dropna()))

In [ ]:
print(dataset.columns)

In [ ]:
#tipos de pagamento/transação
type_unique_values = dataset['type'].unique()
print(type_unique_values)
print(len(type_unique_values))

#Conclusão: podemos(e faz sentido) categorizar esses dados e utilizarmos para treinar nosso classificador

In [ ]:
# origens dos pagamentos
nameOrig_unique_values = dataset['nameOrig'].unique()
print(nameOrig_unique_values)
print(len(nameOrig_unique_values))

#Conclusão: poderíamos utilizar o get_dummies aqui, mas isso faria por adicionar 6353307 colunas
# ao nosso dataset, o que é inviável(acho eu)

In [ ]:
# origens dos pagamentos
nameDest_unique_values = dataset['nameDest'].unique()
print(nameDest_unique_values)
print(len(nameDest_unique_values))

#Conclusão: o mesmo acima vale para o atributo nameDest, vai ser dificil utilizar

In [ ]:
#Boxplot do oldbalanceOrg
print('| Máximo | Mediana | Mínimo | : | {0} | {1} | {2} |'\
      .format(dataset['oldbalanceOrg'].max(), dataset['oldbalanceOrg'].median(), dataset['oldbalanceOrg'].min()))
bplots = plt.boxplot(dataset['oldbalanceOrg'],  vert = 0)

In [ ]:
#Boxplot do newbalanceOrig
print('| Máximo | Mediana | Mínimo | : | {0} | {1} | {2} |'\
      .format(dataset['newbalanceOrig'].max(), dataset['newbalanceOrig'].median(), dataset['newbalanceOrig'].min()))
bplots = plt.boxplot(dataset['newbalanceOrig'],  vert = 0)

In [ ]:
#Boxplot do oldbalanceDest
print('| Máximo | Mediana | Mínimo | : | {0} | {1} | {2} |'\
      .format(dataset['oldbalanceDest'].max(), dataset['oldbalanceDest'].median(), dataset['oldbalanceDest'].min()))
bplots = plt.boxplot(dataset['oldbalanceDest'],  vert = 0)

In [ ]:
#Boxplot do newbalanceDest
print('| Máximo | Mediana | Mínimo | : | {0} | {1} | {2} |'\
      .format(dataset['newbalanceDest'].max(), dataset['newbalanceDest'].median(), dataset['newbalanceDest'].min()))
bplots = plt.boxplot(dataset['newbalanceDest'],  vert = 0)

In [ ]:
# Conclusão a partir dos boxplots dos balanços: posso reduzir o número de colunas, 
# considerar somente o valor que entrou na conta de destino e saiu da conta de origem

In [ ]:
# Boxplot da coluna step
print('| Máximo | Mediana | Mínimo | : | {0} | {1} | {2} |'\
      .format(dataset['step'].max(), dataset['step'].median(), dataset['step'].min()))
bplots = plt.boxplot(dataset['step'],  vert = 0)

In [ ]:
heatmap(dataset.corr())

In [ ]:
#Transformação do dataset, seleção das features

dataset['balanceDeltaOrig'] = original_dataset['newbalanceOrig'] - original_dataset['oldbalanceOrg']
dataset['balanceDeltaDest'] = original_dataset['newbalanceDest'] - original_dataset['oldbalanceDest']

feature_cols = ['amount','oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
target_col = 'isFraud'

dummy_columns = ['type']

prepared_dummy_columns = pd.get_dummies(dataset[dummy_columns])

combined = pd.concat([dataset, prepared_dummy_columns], axis=1)

feature_cols.extend(prepared_dummy_columns.columns.values.tolist())

X = combined[feature_cols]
y = combined[target_col]

X.head()

In [ ]:
#Treino
test_percentage = 0.05
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=test_percentage)

# Criação do classificador de árvore de decisão 
clf = DecisionTreeClassifier(criterion="entropy")

# Usamos o método fit para construir o classificador a partir do nosso conjunto de treinamento
clf = clf.fit(X_train, y_train)

train_accuracy = clf.score(X_train, y_train)

# Usando modelo para classificar os dados que temos a disposição
y_pred = clf.predict(X_test)
test_accuracy = metrics.accuracy_score(y_test, y_pred)

print('| Train accuracy | Test accuracy | : | {0} | {1} |'.format(train_accuracy, test_accuracy))

In [ ]:
print_decision_tree(clf, X.columns)

In [ ]:
#Teste
random_element = random.randint(0, len(X))
sample = X.iloc[[random_element]]
target_of_sample = y.iloc[[random_element]]

pred_sample = clf.predict(sample)

print(pred_sample)
print(target_of_sample)

In [ ]:
def fit_and_test_decision_tree_clf(X, y,criterion, test_percentage, max_depth = 10000):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=test_percentage)

    # Criação do classificador de árvore de decisão 
    classifier = DecisionTreeClassifier(criterion=criterion, max_depth = max_depth)

    # Usamos o método fit para construir o classificador a partir do nosso conjunto de treinamento
    classifier = classifier.fit(X_train, y_train)
    
    train_accuracy = classifier.score(X_train, y_train)

    # Usando modelo para classificar os dados que temos a disposição
    y_pred = classifier.predict(X_test)
    test_accuracy = metrics.accuracy_score(y_test, y_pred)
    
    return classifier, train_accuracy, test_accuracy

# Experimento n:
**Situação Inicial:**
* Classificador **Entropy**
* K-fold 5


In [26]:
kfold = KFold(n_splits=14, shuffle=True, random_state=80)
model_dtc_kfold_3_entropy = DecisionTreeClassifier(criterion="entropy")
print(kfold.get_n_splits(X))

# Médias
accuracy_med_train = 0
accuracy_med_test = 0
count_split = 0

for train_index, test_index in kfold.split(X):
    count_split +=1
    print(" Treino:", train_index, "Teste:", test_index)
    X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    model_dtc_kfold_3_entropy.fit(X_train, y_train)
    y_pred_train = model_dtc_kfold_3_entropy.predict(X_train)
    y_pred_test = model_dtc_kfold_3_entropy.predict(X_test)
    # Cálculo das acurácias
    accuracy_train = metrics.accuracy_score(y_train, y_pred_train)
    accuracy_test = metrics.accuracy_score(y_test, y_pred_test)
    print("Iteração: " + str(count_split) + " \n" + "Acurácia para o treinamento: " + str(accuracy_train) + " \n" + "Acurácia para o teste: " + str(accuracy_test))
    # Cálculo das médias
    accuracy_med_train += accuracy_train/kfold.get_n_splits(X)
    accuracy_med_test += accuracy_test/kfold.get_n_splits(X)

print("Acurácia média para o treinamento:",accuracy_med_train)
print("Acurácia média para o teste:",accuracy_med_test)

14
 Treino: [      0       1       2 ... 6362617 6362618 6362619] Teste: [     17      23      39 ... 6362536 6362600 6362614]
Iteração: 1 
Acurácia para o treinamento: 1.0 
Acurácia para o teste: 0.999786565978617
 Treino: [      0       1       2 ... 6362617 6362618 6362619] Teste: [     15      20      57 ... 6362547 6362562 6362565]
Iteração: 2 
Acurácia para o treinamento: 1.0 
Acurácia para o teste: 0.9997557610683143
 Treino: [      0       1       3 ... 6362617 6362618 6362619] Teste: [      2      40      50 ... 6362559 6362571 6362588]
Iteração: 3 
Acurácia para o treinamento: 1.0 
Acurácia para o teste: 0.999786565978617
 Treino: [      0       1       2 ... 6362616 6362618 6362619] Teste: [      7       9      34 ... 6362586 6362611 6362617]
Iteração: 4 
Acurácia para o treinamento: 1.0 
Acurácia para o teste: 0.9997843656278811
 Treino: [      0       1       2 ... 6362615 6362616 6362617] Teste: [      3       4      19 ... 6362607 6362618 6362619]
Iteração: 5 
Acurácia p